In [18]:
import csv
import os
import random


In [12]:
# ========= CONFIG =========
GOEMOTIONS_PATH = "./goemotions_1/goemotions_merged_clean.csv"      # path to your GoEmotions file
OUTPUT_PATH = "compatika_v1_40mb.txt"   # output dataset
TARGET_SIZE_MB = 40                     # desired size in MB
TARGET_SIZE_BYTES = TARGET_SIZE_MB * 1024 * 1024

In [13]:
# If your labels are text names already; if they are IDs, map IDs -> names here.
# Example emotion set; adjust to match your GoEmotions mapping.
EMOTION_NAMES = [
    "admiration", "amusement", "anger", "annoyance", "approval",
    "caring", "confusion", "curiosity", "desire", "disappointment",
    "disapproval", "disgust", "embarrassment", "excitement", "fear",
    "gratitude", "grief", "joy", "love", "nervousness",
    "optimism", "pride", "realization", "relief", "remorse",
    "sadness", "surprise", "neutral"
]


In [14]:
# Simple templates per emotion (you can expand this later)
TEMPLATES = {
    "sadness": [
        "That sounds really painful. It’s okay to feel sad. Want to share more about what happened?",
        "It makes sense that you’d feel this way. You don’t have to handle it alone. What’s on your mind?"
    ],
    "anger": [
        "It’s understandable to feel angry in a situation like that. Do you want to talk about what set you off?",
        "Your feelings are valid. It sounds like something crossed a line for you."
    ],
    "anxiety": [
        "Feeling anxious can be really exhausting. Want to talk about what’s worrying you most right now?",
        "It’s okay to feel anxious. Taking it one step at a time can help. What’s the biggest thing on your mind?"
    ],
    "fear": [
        "Feeling scared is completely understandable. Do you want to tell me what feels unsafe or uncertain?",
        "It’s okay to feel afraid. You’re not alone in this. What’s making you feel this way?"
    ],
    "joy": [
        "That’s wonderful to hear! What happened that made you feel so happy?",
        "I’m really glad you’re feeling good. Want to tell me more about it?"
    ],
    "love": [
        "That’s such a warm feeling. It’s nice to care about someone that much.",
        "It sounds like this person means a lot to you. That’s really special."
    ],
    "gratitude": [
        "It’s lovely that you feel grateful. What made you feel that way?",
        "That’s so nice. Gratitude can really brighten a day."
    ],
    "neutral": [
        "Thanks for sharing that. Tell me more—what’s on your mind about it?",
        "I hear you. How are you feeling about that right now?"
    ],
    # fallback if no recognized emotion
    "default": [
        "Thank you for sharing that. How are you feeling about it?",
        "I’m listening. Want to tell me more about what’s going on?"
    ],
}

In [15]:

def pick_template(labels):
    priority_order = ["sadness", "anger", "anxiety", "fear", "joy", "neutral"]

    for e in priority_order:
        if e in labels and e in TEMPLATES:
            return random.choice(TEMPLATES[e])

    for e in labels:
        if e in TEMPLATES:
            return random.choice(TEMPLATES[e])

    return random.choice(TEMPLATES["default"])

def load_goemotions(path):
    examples = []
    with open(path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            text = row.get("text", "").strip()
            labels_raw = row.get("labels", "").strip()
            if not text:
                continue
            labels = [l.strip() for l in labels_raw.split(",") if l.strip()]
            examples.append((text, labels))
    return examples


In [16]:

def generate_dataset():
    examples = load_goemotions(GOEMOTIONS_PATH)
    print(f"Loaded {len(examples)} GoEmotions examples.")

    random.shuffle(examples)

    with open(OUTPUT_PATH, "w", encoding="utf-8") as out_f:
        index = 0
        written = 0

        while written < TARGET_SIZE_BYTES:
            text, labels = examples[index]
            response = pick_template(labels)

            sample = f"USER: {text}\nCOMPATIKA: {response}\n\n"

            out_f.write(sample)
            written += len(sample.encode("utf-8"))

            index += 1
            if index >= len(examples):
                index = 0

    final_size = os.path.getsize(OUTPUT_PATH)
    print(f"Done! Output file: {OUTPUT_PATH}, size: {final_size / (1024*1024):.2f} MB")

In [19]:
generate_dataset()

Loaded 211225 GoEmotions examples.
Done! Output file: compatika_v1_40mb.txt, size: 40.80 MB


In [22]:
import csv
import os
import random

# ================== CONFIG ==================
# 👇 Change this to the FULL path of your GoEmotions CSV
GOEMOTIONS_PATH = "./goemotions_1/goemotions_merged_clean.csv" 

OUTPUT_PATH = "compatika_v1_40mb.txt"
TARGET_SIZE_MB = 40
TARGET_SIZE_BYTES = TARGET_SIZE_MB * 1024 * 1024

# ================== TEMPLATES ==================
# Compatika-style emotional responses.
TEMPLATES = {
    "sadness": [
        "It sounds like you’re carrying a lot of sadness. You don’t have to go through it alone. Want to talk about what hurts?",
        "Your feelings make sense. When something matters to us, sadness can hit deeply. What part feels the heaviest right now?",
        "It’s okay to feel this way. I’m here with you. What happened that brought these feelings up?",
        "It sounds really painful. You deserve space to feel this. Want to share more about what’s going on?"
    ],
    "anger": [
        "It’s understandable to feel angry when things feel unfair. Do you want to talk through what happened?",
        "Your anger makes sense. Something important must have felt crossed. Want to tell me about it?",
        "It sounds like this really got to you. I’m here to listen if you want to unpack it.",
        "Feeling frustrated doesn’t make you wrong. It means something didn’t sit right. What part bothered you the most?"
    ],
    "anxiety": [
        "Feeling anxious can be exhausting. I’m here with you. Want to talk about what’s worrying you right now?",
        "It’s okay to feel nervous. You’re doing your best. What part of this situation feels the most overwhelming?",
        "Anxiety can show up fast. Take your time. What’s the biggest thought on your mind?",
        "You’re not alone in feeling this way. What would help you feel a little calmer?"
    ],
    "nervousness": [  # alias for anxiety
        "It’s completely okay to feel nervous. New or uncertain things can be scary. What’s making you feel this way?",
        "It sounds like you’re on edge about this. Do you want to talk through what you’re worried might happen?"
    ],
    "fear": [
        "It’s completely okay to feel scared. Something must feel uncertain. Want to share what’s making you feel this way?",
        "Fear can be hard to carry alone. I’m here with you. What’s the part that feels the most frightening?",
        "It sounds like you’re facing something difficult. What would help you feel a bit safer right now?",
        "Your fear is valid. It’s okay to take things one small step at a time."
    ],
    "joy": [
        "That’s wonderful to hear! What happened that made your day so good?",
        "I’m really glad you’re feeling good. Want to tell me more about it?",
        "That sounds really joyful. It’s nice to hear moments like this. What made it special?",
        "I love that you’re feeling this way. What’s the highlight for you right now?"
    ],
    "love": [
        "That’s such a warm feeling. It’s nice to care about someone that much.",
        "It sounds like this person means a lot to you. That’s really special.",
        "Love can be powerful and tender at the same time. How does it feel to carry that right now?",
        "It’s beautiful that you feel this way. Would you like to talk more about them?"
    ],
    "gratitude": [
        "It’s lovely that you feel grateful. What made you feel that way?",
        "That’s so nice. Gratitude can really brighten a day.",
        "It sounds like you really appreciate this. What part of it matters most to you?",
        "Noticing things you’re grateful for can be really grounding. What are you grateful for today?"
    ],
    "disappointment": [
        "It makes sense to feel disappointed when things don’t go how you hoped. Want to talk about what happened?",
        "That sounds really discouraging. Your feelings are valid. What did you wish had gone differently?",
        "It’s okay to feel let down. Sometimes plans or people don’t match our hopes.",
        "I hear your disappointment. What would have felt better for you in this situation?"
    ],
    "remorse": [
        "It sounds like you’re really reflecting on what happened. Regret can be heavy. Want to talk about it?",
        "The fact that you feel remorse shows you care. Would it help to think about how you might move forward?",
        "It’s okay to wish you’d done things differently. You’re allowed to learn and grow from this.",
        "You’re not alone in feeling this way. What would being kinder to yourself look like here?"
    ],
    "surprise": [
        "That sounds surprising. How are you feeling about it now that it’s happened?",
        "Wow, that seems unexpected. Was it a pleasant surprise or more unsettling?",
        "It can be a lot when things catch us off guard. What part surprised you the most?",
        "Unexpected moments can bring up mixed feelings. How is it sitting with you now?"
    ],
    "amusement": [
        "That does sound amusing. I’m glad you had a light moment like that. Want to share more?",
        "It’s nice to find things that make you laugh. What about it made you smile?",
        "I love that you found that funny. Little moments like that can really brighten a day.",
        "It’s great that you can laugh about it. Humor can be such a relief."
    ],
    "caring": [
        "It’s really kind that you care so much. How are you feeling about the situation?",
        "Your care really shows. It sounds like this person matters a lot to you.",
        "It’s thoughtful of you to be so concerned. How can you take care of yourself too?",
        "Caring deeply can be both beautiful and heavy. What’s on your heart right now?"
    ],
    "confusion": [
        "It’s okay to feel confused. Things don’t always make sense right away. Want to talk it through?",
        "Confusion can feel uncomfortable, but it’s also part of figuring things out. What’s most unclear for you?",
        "You don’t have to have all the answers. What part is bothering you the most?",
        "It sounds like you’re trying to make sense of a lot. I’m here if you want to unpack it slowly."
    ],
    "curiosity": [
        "It’s great that you’re curious. What would you like to explore or understand more?",
        "Curiosity can lead to meaningful discoveries. What’s on your mind?",
        "I like that you’re asking questions. Where do you feel like starting?",
        "Your curiosity is a strength. What are you most interested in right now?"
    ],
    "desire": [
        "It sounds like this is really important to you. What draws you to it the most?",
        "Wanting something deeply can feel intense. How does it feel to sit with that desire?",
        "It’s okay to want what you want. What would it mean to you to have it?",
        "That longing sounds strong. Do you want to talk about it more?"
    ],
    "disgust": [
        "It sounds like this really pushed your boundaries. Want to tell me what felt so off about it?",
        "Feeling disgusted can be a strong signal that something doesn’t align with your values.",
        "Your reaction is valid. Something in this clearly didn’t sit right with you.",
        "It’s okay to feel repulsed or turned off. Do you want to share more about why?"
    ],
    "embarrassment": [
        "Feeling embarrassed can be really uncomfortable. You’re not alone in that.",
        "It sounds like you’re being hard on yourself. Many people would feel the same in your place.",
        "That must have felt awkward. Do you want to talk about what happened?",
        "It’s okay to cringe at things sometimes. It doesn’t define who you are."
    ],
    "excitement": [
        "That’s exciting! It sounds like something really positive is happening. Want to share more?",
        "I love hearing your excitement. What are you looking forward to the most?",
        "It’s great that you feel so energized about this. What’s making it special?",
        "Your excitement really comes through. How does it feel to anticipate this?"
    ],
    "grief": [
        "I’m really sorry you’re going through this. Grief can be incredibly heavy.",
        "It sounds like you’ve lost something or someone important. I’m here with you.",
        "Grief can come in waves. However it shows up is okay. Want to talk about what you’re missing?",
        "Your pain is valid. You don’t have to rush through it. I’m here to listen."
    ],
    "optimism": [
        "It’s lovely that you’re feeling hopeful. What makes you feel optimistic right now?",
        "That’s a really encouraging outlook. What are you looking forward to?",
        "Holding onto hope can be powerful. What helps you stay positive?",
        "Your optimism really shines through. It’s nice to hear that."
    ],
    "pride": [
        "That’s amazing—you have every right to feel proud. What did you do that you’re celebrating?",
        "It’s great that you’re recognizing your own effort. You earned that feeling.",
        "I’m glad you’re giving yourself credit. What does this achievement mean to you?",
        "You should be proud of yourself. It sounds like you worked hard for this."
    ],
    "realization": [
        "That sounds like an important realization. How does it feel to see things this way now?",
        "New insights can be powerful, even if they’re uncomfortable. What stands out the most to you?",
        "It seems like something has really clicked for you. Want to talk more about it?",
        "Realizations like that can shift a lot. How are you processing it?"
    ],
    "relief": [
        "I’m glad you feel some relief. That must take a bit of weight off.",
        "It sounds like a burden has lightened. What feels different now?",
        "Relief can be such a comforting feeling. What changed for you?",
        "I’m happy to hear things feel a bit easier. How are you feeling now compared to before?"
    ],
    "approval": [
        "It sounds like you feel good about this, and that’s important.",
        "I’m glad this feels right to you. Your perspective matters.",
        "It’s nice that this lines up with your values. How does that feel?",
        "Your sense of approval here really shows what you care about."
    ],
    "disapproval": [
        "It sounds like this really didn’t sit well with you. Want to talk about why?",
        "Your discomfort makes sense. Something here must feel off or wrong.",
        "It’s okay to disagree or disapprove. What part feels most important to you?",
        "Your reaction shows that your values matter a lot to you."
    ],
    "admiration": [
        "It’s really nice that you admire this. What about it inspires you?",
        "That kind of appreciation says a lot about what you value.",
        "It sounds like this person or thing really resonates with you. What stands out?",
        "Admiration like that can be really motivating. How does it influence you?"
    ],
    "neutral": [
        "Thanks for sharing that. How are you feeling about it?",
        "I hear you. What’s on your mind about this right now?",
        "That makes sense. Is there anything you’d like to explore about it?",
        "I’m listening. Tell me more, if you’d like."
    ],
    "default": [
        "Thank you for sharing that. How are you feeling about it?",
        "I’m listening. Want to tell me more about what’s going on?",
        "Your perspective matters. What feels most important to you right now?",
        "I’m here with you. What would you like to talk about more?"
    ],
}

# Map raw GoEmotions labels to our template keys (canonical names)
EMOTION_CANONICAL = {
    "sadness": "sadness",
    "anger": "anger",
    "annoyance": "anger",
    "nervousness": "anxiety",
    "anxiety": "anxiety",
    "fear": "fear",
    "joy": "joy",
    "love": "love",
    "gratitude": "gratitude",
    "disappointment": "disappointment",
    "remorse": "remorse",
    "grief": "grief",
    "surprise": "surprise",
    "amusement": "amusement",
    "caring": "caring",
    "confusion": "confusion",
    "curiosity": "curiosity",
    "desire": "desire",
    "disgust": "disgust",
    "embarrassment": "embarrassment",
    "excitement": "excitement",
    "optimism": "optimism",
    "pride": "pride",
    "realization": "realization",
    "relief": "relief",
    "approval": "approval",
    "disapproval": "disapproval",
    "admiration": "admiration",
    "neutral": "neutral",
}

PRIORITY_ORDER = [
    "grief", "sadness", "remorse", "fear", "anxiety",
    "anger", "disappointment", "disgust",
    "joy", "excitement", "love", "gratitude", "optimism", "pride",
    "caring", "confusion", "curiosity",
    "relief", "surprise", "amusement",
    "approval", "disapproval", "admiration",
    "neutral"
]

def pick_template(labels):
    """
    Pick a Compatika-style reply based on emotion labels from GoEmotions.
    labels = list of raw label strings, e.g. ["sadness", "fear"]
    """
    # Normalize labels -> canonical keys
    canon_labels = []
    for l in labels:
        l = l.strip()
        if l in EMOTION_CANONICAL:
            canon_labels.append(EMOTION_CANONICAL[l])

    if not canon_labels:
        return random.choice(TEMPLATES["default"])

    # Try priority order first
    for emo in PRIORITY_ORDER:
        if emo in canon_labels and emo in TEMPLATES:
            return random.choice(TEMPLATES[emo])

    # Fallback: any matching template
    for emo in canon_labels:
        if emo in TEMPLATES:
            return random.choice(TEMPLATES[emo])

    # Final fallback
    return random.choice(TEMPLATES["default"])

# ================== DATA LOADING & GENERATION ==================

def load_goemotions(path):
    """
    Load GoEmotions CSV.
    Assumes columns: 'text' and 'labels'
    where 'labels' is like 'sadness,joy' or 'sadness'.
    Adjust column names if your file is different.
    """
    examples = []
    with open(path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            text = row.get("text", "").strip()
            labels_raw = row.get("labels", "").strip()
            if not text:
                continue
            labels = [l.strip() for l in labels_raw.split(",") if l.strip()]
            examples.append((text, labels))
    return examples

def generate_dataset():
    examples = load_goemotions(GOEMOTIONS_PATH)
    print(f"Loaded {len(examples)} GoEmotions examples.")

    random.shuffle(examples)

    with open(OUTPUT_PATH, "w", encoding="utf-8") as out_f:
        index = 0
        written = 0

        while written < TARGET_SIZE_BYTES:
            text, labels = examples[index]
            response = pick_template(labels)

            sample = f"USER: {text}\nCOMPATIKA: {response}\n\n"

            out_f.write(sample)
            written += len(sample.encode("utf-8"))

            index += 1
            if index >= len(examples):
                index = 0

    final_size = os.path.getsize(OUTPUT_PATH)
    print(f"Done! Output file: {OUTPUT_PATH}, size: {final_size / (1024*1024):.2f} MB")


In [23]:
generate_dataset()


Loaded 211225 GoEmotions examples.
Done! Output file: compatika_v1_40mb.txt, size: 40.79 MB
